In [1]:
import os
import torch
import numpy as np
# from networks.Attn_SN_network import Generator
# from networks.SN_network import Generator
from networks.CNN_network import Generator
# from networks.ResNet_network import Generator
from utils.utils import denorm

# CIFAR10

In [2]:
# save_path = './output/'

# dataset = 'cifar'
# image_path = '~/Datasets/CIFAR10/'
# imsize = 32

# STL-10

In [3]:
save_path = './output/'

dataset = 'stl10'  # 'mnist', 'cifar', 'lsun', 'celeb', 'stl10'
image_path = '~/Datasets/STL-10/'  # '/media/v/data1/MNIST', '/media/v/data1/cifar10/', '/media/v/data1/LSUN', '/media/v/data1/CelebA'
imsize = 96

In [4]:
model_save_path = save_path + '/models'

channel = 3
z_dim = 64
g_conv_dim = 64

## 1. Compute Inception Score

In [5]:
# from utils.inception_score import get_inception_score

# pretrained_model = 300000

# # fake_images (get with CPU)
# z = torch.randn(samples_num, z_dim)#.cuda()
# G = Generator(imsize, z_dim, channel, g_conv_dim)#.cuda()
# G.load_state_dict(torch.load(os.path.join(model_save_path, 'G_step{}.pth'.format(pretrained_model))))
# with torch.no_grad():
#     fake_images = G(z) #[-1, 1]
    
# (mean, std) = get_inception_score(denorm(fake_images).mul(255).clamp(0, 255).byte().permute(0,2,3,1).numpy())

In [6]:
# print('Inception Score: ' + str(round(mean,2)) + '±' + str(round(std,2)))

## 2. Compute fid

In [7]:
from utils.fid_pytorch import get_fid_pytorch

pretrained_model =  [200000]#, 40000, 60000, 80000, 100000]
FIDs = []

print('iters\t\t\tFID')

for i in range(len(pretrained_model)):
    # fake_images
    fake_images = np.empty((0, channel, imsize, imsize))
    G = Generator(imsize, z_dim, channel, g_conv_dim).cuda()
    G.load_state_dict(torch.load(os.path.join(model_save_path, 'G_step{}.pth'.format(pretrained_model[i]))))
    for j in range(20): # generate 50K
        z = torch.randn(1000, z_dim).cuda()        
        with torch.no_grad():
            fake_image_2K = G(z).cpu().numpy() #[-1, 1]
        fake_images = np.vstack((fake_images, fake_image_2K))
    
    
    # real_images
    from utils.data_loader import Data_Loader 
    data_loader = Data_Loader(dataset = dataset, image_path = image_path, image_size = imsize, batch_size = 20000, shuf = True)
    data_iter = iter(data_loader.loader())
    real_images, _ = next(data_iter)
    
    fid_pytorch = get_fid_pytorch(real_images.numpy(), fake_images.astype(np.float32))
    FIDs.append(fid_pytorch)
    
    print(str(pretrained_model[i]) + '\t\t' + str(round(fid_pytorch,2)))

iters			FID
200000		51.67
